In [2]:
# Importing libraries 
from datetime import datetime
import os
import pandas as pd
import sys
sys.path.append(r"C:\Users\Alireza\Documents\Git\Cosine-Project")
from inSilico_experiments.utils.func_lib import *
import matplotlib.pylab as plt
##
import warnings
warnings.filterwarnings("ignore") # becairfull with this
#%%
# parameters parsing
# 
NUM_OF_RUNS_PER_POP_SIZE = 1
NUM_OF_RUNS_PER_IMAGE = 1
NUM_OF_RUN_RER_SUBPOAP = 1

#%%
# set main fynction
if __name__=="__main__":
    #%%
    #import datetime
    now = datetime.now()
    from core.utils.GAN_utils import upconvGAN
    from core.utils.Optimizers import CholeskyCMAES
    from core.utils.grad_RF_estim import fit_2dgauss, GAN_grad_RF_estimate
    from core.utils.CNN_scorers import TorchScorer, resize_and_pad_tsr
    #%% 
    # Set parameters
    date_time_str = now.strftime("%Y-%m-%d_%H-%M")
    refimgdir = r"C:\Data\cosine\insilico_experiments\data\image_net_tiny"
    exp_result_root = r"C:\Data\cosine\insilico_experiments\Most_active_vs_random\results\rotated_%s" % date_time_str
    os.makedirs(exp_result_root, exist_ok=True)

    net_name = "alexnet"
    layer_name = ".features.Conv2d10"
    layer_short = "conv5"
    gan_name = "fc6"
    pop_size = [16, 64, 128, 256]
    metric_name = ["cosine", "MSE"]
    input_size = (3, 227, 227)
    img_size = (187, 187)
    pading_size = (20, 20) 
    #%% make a dictionary of the parameters
    param_dict = {"net_name": net_name,
                    "layer_name": layer_name,
                    "layer_short": layer_short,
                    "gan_name": gan_name,
                    "pop_size": pop_size,
                    "input_size": input_size,
                    "pading_size": pading_size}
    trial_param_dict = param_dict.copy()
    #%% set up a pandas dataframe to save the inforamtions about each run
    expriment_meta_data_df = pd.DataFrame(columns=["output_type", "trget_imge_name", "similarity_metric", "pop_size", "pop_resampling_id",
                                       "gan_name", "layer_name", "layer_short", "net_name", "img_size", "pading_size", "input_size", "score","pop_unit_idx",
                                       "gen_rerun_id", "sub_pop_type", "sub_pop_size", "sub_pop_re_run_id", "save_dir_root", "file_name"])

    #%% save the parameters
    #%% Load the reference images
    refimgnms, refimgtsr = load_ref_imgs(
        imgdir=refimgdir, preprocess_type='center_crop', image_size=227)
    #%% Set up the scorer and the population
    scorer = TorchScorer(net_name)
    module_names, module_types, module_spec = get_module_names(
        scorer.model, input_size, "cuda", False)

    # %% set up the GAN
    G = upconvGAN(gan_name).cuda()
    G.requires_grad_(False)
    code_length = G.codelen
    
    #%% Run the experiment
    scorer = TorchScorer(net_name)

    unit_mask_dict, unit_tsridx_dict = set_random_population_recording(
                scorer, [layer_name], randomize=False)
    # Encode a population of images to set the normalizer and mask.
    ref_actmat, _ = encode_image(scorer, refimgtsr, key=layer_name,
                            RFresize=True, corner=pading_size, imgsize=img_size)    
   

Layer .features.Conv2d10 Sampled 256 units from feature tensor of shape (256, 13, 13)


In [12]:

def set_most_active_population_recording(scorer_most, targetnames, refimgtsr, corner_pading, img_size, popsize=500, single_col=True, resample=False,
                                    seed=None):
    most_active_unit_tsridx_dict = {}
    most_active_unit_mask_dict = {}
    module_names, module_types, module_spec = get_module_names(scorer_most.model, (3,227,227), "cuda", False)
    invmap = {v: k for k, v in module_names.items()}
    try:
        for layer in targetnames:
            outshape = module_spec[invmap[layer]]["outshape"]

            flat_idx_all = sample_center_column_units_idx(outshape, single_col=True)
            scorer_most.set_popul_recording(layer, flat_idx_all, )
            ref_actmat, _ = encode_image(scorer_most, refimgtsr, key=layer,
                            RFresize=True, corner=corner_pading, imgsize=img_size)   
            # get popsiz elements largest values of ref_actmat
            most_actve_unit_idx = np.argsort(ref_actmat, axis=None)[-popsize:] 
            flat_idx_most_actve = flat_idx_all[most_actve_unit_idx]

            scorer_most.cleanup(print_info=False)
        
            tsr_idx_most_active = np.unravel_index(flat_idx_most_actve, outshape)
            most_active_unit_mask_dict[layer] = flat_idx_most_actve
            most_active_unit_tsridx_dict[layer] = tsr_idx_most_active
            scorer_most.set_popul_recording(layer, flat_idx_most_actve, )

    except KeyError:
        print(*invmap.keys(), sep="\n")
        raise KeyError
    return most_active_unit_mask_dict, most_active_unit_tsridx_dict


def set_least_active_population_recording(scorer_least, targetnames, refimgtsr, corner_pading, img_size, popsize=500, single_col=True, resample=False,
                                    seed=None):
    least_active_unit_tsridx_dict = {}
    least_active_unit_mask_dict = {}
    module_names, module_types, module_spec = get_module_names(scorer_least.model, (3,227,227), "cuda", False)
    invmap = {v: k for k, v in module_names.items()}
    try:
        for layer in targetnames:
            outshape = module_spec[invmap[layer]]["outshape"]

            flat_idx_all = sample_center_column_units_idx(outshape, single_col=True)
            scorer_least.set_popul_recording(layer, flat_idx_all, )
            ref_actmat, _ = encode_image(scorer_least, refimgtsr, key=layer,
                            RFresize=True, corner=corner_pading, imgsize=img_size)   
            # get popsiz elements smallest values of ref_actmat
            least_actve_unit_idx = np.argsort(ref_actmat, axis=None)[:popsize]
            
            flat_idx_least_actve = flat_idx_all[least_actve_unit_idx]
            
            scorer_least.cleanup(print_info=False)
        
            tsr_idx_most_active = np.unravel_index(flat_idx_least_actve, outshape)
            least_active_unit_mask_dict[layer] = flat_idx_least_actve
            least_active_unit_tsridx_dict[layer] = tsr_idx_most_active
            scorer_least.set_popul_recording(layer, flat_idx_least_actve, )

    except KeyError:
        print(*invmap.keys(), sep="\n")
        raise KeyError
    return least_active_unit_mask_dict, least_active_unit_tsridx_dict
#%% let test the function
imgid = 0
scorer = TorchScorer(net_name)
unit_mask_dict, unit_tsridx_dict, ref_actmat, most_actve_unit_idx = set_least_active_population_recording(scorer, [layer_name], refimgtsr[imgid:imgid + 1], pading_size, img_size, popsize=128)


In [13]:
ref_actmat_best, _ = encode_image(scorer, refimgtsr, key=layer_name,
                            RFresize=True, corner=pading_size, imgsize=img_size)   

In [14]:
# print ref_actmat_best sorted
np.sort(ref_actmat_best[0,:])
#%

array([-18.43441  , -18.400341 , -18.011045 , -17.981722 , -17.616796 ,
       -17.582958 , -17.37425  , -17.137459 , -17.0347   , -16.739782 ,
       -16.04295  , -15.9975195, -15.690247 , -15.441564 , -15.19063  ,
       -14.986549 , -14.976959 , -14.661179 , -14.627042 , -14.396803 ,
       -14.285725 , -14.157491 , -13.895635 , -13.751593 , -13.669177 ,
       -13.331209 , -13.30969  , -13.302381 , -13.201684 , -13.070899 ,
       -12.98473  , -12.928744 , -12.927791 , -12.917333 , -12.863702 ,
       -12.85152  , -12.797337 , -12.590782 , -12.555866 , -12.500052 ,
       -12.454038 , -12.431001 , -12.405144 , -12.327293 , -12.161742 ,
       -12.108588 , -12.048702 , -11.946264 , -11.92395  , -11.861829 ,
       -11.843388 , -11.832071 , -11.715294 , -11.714884 , -11.660082 ,
       -11.403873 , -11.336632 , -11.31639  , -11.2299385, -11.207528 ,
       -11.133642 , -11.080625 , -10.898127 , -10.746104 , -10.690709 ,
       -10.67484  , -10.609883 , -10.559575 , -10.558177 , -10.5

In [15]:
ref_actmat[0,most_actve_unit_idx]

array([-18.434404 , -18.400337 , -18.01104  , -17.981714 , -17.61679  ,
       -17.582958 , -17.374247 , -17.137453 , -17.034695 , -16.739786 ,
       -16.04294  , -15.9975195, -15.690246 , -15.44156  , -15.190626 ,
       -14.9865465, -14.976962 , -14.66117  , -14.6270485, -14.396799 ,
       -14.285722 , -14.157482 , -13.895637 , -13.751583 , -13.669168 ,
       -13.331206 , -13.309689 , -13.302379 , -13.201688 , -13.070895 ,
       -12.984735 , -12.928741 , -12.927787 , -12.917329 , -12.863696 ,
       -12.851512 , -12.79734  , -12.59078  , -12.555859 , -12.500049 ,
       -12.454038 , -12.431    , -12.40514  , -12.327299 , -12.161738 ,
       -12.108587 , -12.048702 , -11.946255 , -11.923947 , -11.861823 ,
       -11.8433895, -11.832066 , -11.715292 , -11.71488  , -11.660072 ,
       -11.4038725, -11.336632 , -11.316391 , -11.229938 , -11.207526 ,
       -11.133646 , -11.080623 , -10.89812  , -10.746106 , -10.690702 ,
       -10.674842 , -10.609886 , -10.559575 , -10.558171 , -10.5

In [ ]:
 for popsize in pop_size:
        #%% add the population size to the dictionary
        trial_param_dict = param_dict.copy()
        trial_param_dict["pop_size"] = popsize
        trial_param_dict["sub_pop_type"] = "whole"
        #%% Set the population
        for pop_resampling_id in range(NUM_OF_RUNS_PER_POP_SIZE):
            # add the population resampling id to the dictionary
            trial_param_dict["pop_resampling_id"] = pop_resampling_id
            # Set scorer and select population    
            
            unit_mask_dict, unit_tsridx_dict = set_random_population_recording(
                scorer, [layer_name], popsize=popsize, seed=pop_resampling_id)
            ## add the population unit index to the dictionary
            trial_param_dict["pop_unit_idx"] = unit_mask_dict[layer_name]
            # Encode a population of images to set the normalizer and mask.
            ref_actmat, _ = encode_image(scorer, refimgtsr, key=layer_name,
                                    RFresize=True, corner=pading_size, imgsize=img_size)
            popul_m, popul_s = set_normalizer(ref_actmat)
            popul_mask = set_popul_mask(ref_actmat)
            # receptive field estimation
            unitslice = (unit_tsridx_dict[layer_name][0],
                        unit_tsridx_dict[layer_name][1][0],
                        unit_tsridx_dict[layer_name][2][0])
            gradAmpmap = GAN_grad_RF_estimate(G, scorer.model, layer_name, unitslice, input_size=input_size,
                                        device="cuda", show=False, reps=100, batch=1)
            fitdict = fit_2dgauss(gradAmpmap)